In [1]:
import numpy as np
import pandas as pd
import os
import glob
import shutil
from osgeo import gdal
import struct
import utm
import matplotlib.pyplot as plt

In [2]:
video_name = 'observation088'
drone_log_folder = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/drone-flightlogs/' + video_name + '/'
drone_logs = glob.glob(drone_log_folder + '*')
drone_logs.sort()
frame_folders_root = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-frames/raw-footage/observation088' 
output_folder_root = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/processed-videos/raw-footage/observation088'

video_frame_folders = glob.glob(frame_folders_root + '/*')
video_frame_folders.sort()

In [4]:
# Get first and last frame number for every video segment in each drone flight

first_last_folders = []
video_ind = -1
for folder_ind, folder in enumerate(video_frame_folders):
    files = glob.glob(folder + '/*.jpg')
    files.sort(key=lambda file: int(file.split('_')[-1].split('.')[0]))

    first_frame = int(files[0].split('_')[-1].split('.')[0])
    last_frame = int(files[-1].split('_')[-1].split('.')[0])
    if first_frame == 0:
        first_last_folders[video_ind].append({'first_frame': first_frame, 'last_frame': last_frame})
    else:
        video_ind += 1
        first_last_folders.append([])
        first_last_folders[video_ind].append({'first_frame': first_frame, 'last_frame': last_frame})

In [5]:
first_last_folders

[[{'first_frame': 6246, 'last_frame': 19630},
  {'first_frame': 0, 'last_frame': 19634},
  {'first_frame': 0, 'last_frame': 19632},
  {'first_frame': 0, 'last_frame': 8226}],
 [{'first_frame': 7624, 'last_frame': 19630},
  {'first_frame': 0, 'last_frame': 19634},
  {'first_frame': 0, 'last_frame': 19632},
  {'first_frame': 0, 'last_frame': 12094}],
 [{'first_frame': 11228, 'last_frame': 19628},
  {'first_frame': 0, 'last_frame': 19634},
  {'first_frame': 0, 'last_frame': 19634},
  {'first_frame': 0, 'last_frame': 9536}]]

In [6]:
# Get the first and last frame number for drone flight (merging all video segments in drone flight together)

first_last_flights = []

for folder_list in first_last_folders:
    first_frame = folder_list[0]['first_frame']
    last_frame = folder_list[0]['last_frame']
    for fl_dict in folder_list[1:]:
            last_frame += fl_dict['last_frame']
        
    first_last_flights.append({'first_frame': first_frame, 'last_frame': last_frame})

In [7]:
first_last_flights

[{'first_frame': 6246, 'last_frame': 67122},
 {'first_frame': 7624, 'last_frame': 70990},
 {'first_frame': 11228, 'last_frame': 68432}]

In [17]:
flight_logs = []
for file in drone_logs:
    flight_log = pd.read_csv(file)
    # Get rid of potential white space in column names
    flight_log.rename(columns=lambda x: x.strip(), inplace=True)
    flight_logs.append(flight_log)

In [18]:
num_flights = len(drone_logs)
# These have to be manually gotten from the videos or frame extractions
first_moving_frames = [799, 524, 1311]

if len(first_moving_frames) != num_flights:
    raise ValueError('make sure you have a value that ties every flight to the drone flight logs')

In [20]:
# Find the first entries in the drone flight logs where there is movement recorded
# This is assumed to correspond in the time with the first movement seen in the frames extracted
# from the video in the cell above.
# This correspondence is used to link the frames in the video to the information in 
# in the flight logs for the entire flight

first_move = [np.min(np.nonzero(flight_logs[flight_ind].loc[:, 'ascent(feet)'])) for flight_ind in range(num_flights)]
print(first_move)
#  Add estimated frame numbers of the video to the flight log
for df_ind, df in enumerate(flight_logs):
    df.loc[:, 'frame_num'] = (
        (df.loc[:, 'time(millisecond)'] - df.loc[first_move[df_ind], 'time(millisecond)']) / 16.666).astype(int) + first_moving_frames[df_ind]

[88, 45, 32]


In [21]:
# Go through every entry and record frames where drone has moved by a certrain threshold from previous recording
# Use these frames to feed pix4d to create map of space and get ground truth drone movement

# Get dataframe row numbers and frame numbers of images to use for map construction

drone_movement_threshold = 0.00005

frame_list = [[] for _ in range(3)]
ind_list = [[] for _ in range(3)]
for flight_ind in range(3):
# flight_ind = 0 
    # index in flight log coresponding to first frame in recording
    first_ind = np.min(np.nonzero(flight_logs[flight_ind]['frame_num'] >= first_last_flights[flight_ind]['first_frame']))
    # index in flight log coresponding to last frame in recording
    last_ind = np.min(np.nonzero(flight_logs[flight_ind]['frame_num'] >= first_last_flights[flight_ind]['last_frame']))

    # record the entries in the flight log the correspond to frames in the part of the video
    # over the animals

    flight_logs[flight_ind] = flight_logs[flight_ind].assign(observing = False)
    flight_logs[flight_ind].loc[first_ind:last_ind, 'observing'] = True
    
    # For images used in pix4d map, save the actual image name, none if not used in map
    flight_logs[flight_ind] = flight_logs[flight_ind].assign(image_name = None)

    # Record the frames that are used to create the pix4d map

    flight_logs[flight_ind] = flight_logs[flight_ind].assign(used_in_map = False)

    # log = flight_logs[flight_ind].loc[first_ind:last_ind]
    log = flight_logs[flight_ind]

    ind_list[flight_ind].append(first_ind)
    frame_list[flight_ind].append(log.loc[first_ind, 'frame_num'])
    flight_logs[flight_ind].loc[first_ind, 'used_in_map'] = True

    last_coord = np.array([log.loc[first_ind, 'latitude'], log.loc[first_ind, 'longitude']])
    last_saved_row_ind = first_ind
    last_heading = log.loc[first_ind, 'compass_heading(degrees)']
    # Is the drone curently rotating
    rotating = False
    # Is the drone currently moving
    moving = False
    # How often to take frame when it is already rotating
    rotating_thresh = 30
    # How many degrees the drone must turn for a rotation to begin
    start_rotating_thresh = 1
    # Speed theshold for moving
    start_moving_thresh = 1.5

    rotate_count = 0
    move_count = 0
    thresh_count = 0
    
    heading_last_row = log.loc[first_ind, 'compass_heading(degrees)']


    for ind, row in log.loc[log['observing'] == True].iterrows():
        save_frame = False
        cur_coord = np.array([row['latitude'], row['longitude']])
        heading_dif = np.abs(row['compass_heading(degrees)'] - last_heading)
        if not rotating:
            if heading_dif > start_rotating_thresh:
                rotating = True
                save_frame = True
                rotate_count += 1
        elif rotating:
            if heading_dif > rotating_thresh:
                save_frame = True
                rotate_count += 1
            elif row['compass_heading(degrees)'] - heading_last_row == 0:
                rotating = False
                save_frame = True
                rotate_count += 1
        heading_last_row = row['compass_heading(degrees)']
        if not moving and row['speed(mph)'] > start_moving_thresh:
            save_frame = True
            moving = True
            move_count += 1
        if moving and row['speed(mph)'] == 0:
            save_frame = True
            moving = False
            move_count += 1


        diff = np.sqrt(np.sum(np.square(cur_coord - last_coord)))
        if diff > drone_movement_threshold:
            save_frame = True
            thresh_count += 1

        if save_frame:
            frame_list[flight_ind].append(row['frame_num'])
            ind_list[flight_ind].append(ind)
            flight_logs[flight_ind].loc[ind, 'used_in_map'] = True
            last_coord = cur_coord
            last_heading = row['compass_heading(degrees)']


    ind_list[flight_ind].append(last_ind)
    frame_list[flight_ind].append(log.loc[last_ind, 'frame_num'])
    flight_logs[flight_ind].loc[last_ind, 'used_in_map'] = True

    print(len(frame_list[flight_ind]))
    # print(list(flight_logs[flight_ind].loc[flight_logs[flight_ind]['used_in_map'], 'frame_num']))
    print('rotate count', rotate_count)
    print('move count', move_count)
    print('thresh count', thresh_count)
    
print([len(l) for l in frame_list])
print(sum([len(l) for l in frame_list]))


34
rotate count 3
move count 18
thresh count 11
64
rotate count 14
move count 36
thresh count 12
88
rotate count 2
move count 28
thresh count 57
[34, 64, 88]
186


In [22]:
flight_ind = 2
# print(list(flight_logs[flight_ind].loc[flight_logs[flight_ind]['observing'], 'observing']))
list(flight_logs[flight_ind].loc[flight_logs[flight_ind]['observing'], 'used_in_map'])[2]

False

[[6247, 23941, 24139, 24331, 24517, 24715, 24913, 27458, 33950, 34208, 48392, 48578, 55551, 62643, 62739, 67125], [], []]

In [28]:
# Get the actual frames chosen above to construct map

SAVE_IMAGES = False

video_frame_folders = glob.glob(frame_folders_root + '/*')
video_names = [folder.rpartition('_')[0] for folder in video_frame_folders]
# have only one entry for every video name
video_names = list(set(video_names))
video_names.sort()
# print('video names', video_names)

map_images_folder = os.path.join(output_folder_root, 'map-images/whole-observation')
if not os.path.isdir(map_images_folder):
    os.mkdir(map_images_folder)

video_ind = 0
export_images = []
for video_ind in range(3):
    image_folders = glob.glob(video_names[video_ind] + '*')
    image_folders.sort()

    


    fl_dicts = first_last_folders[video_ind]
    for map_frame_ind, frame_num in enumerate(frame_list[video_ind]):
        ex_frame_num = 0
        for folder_ind, fl_dict in enumerate(fl_dicts):
            if frame_num <= fl_dict['last_frame']:
                # make frame num even since those are the only frames that get extracted
                if frame_num % 2 != 0:
                    frame_num -= 1
                image_file = glob.glob(image_folders[folder_ind] + '/*_*' + str(frame_num) + '.jpg')
                export_images.append(image_file[0])
                if not os.path.exists(image_file[0]):
                    print('following file does not exist: {}'.format(image_file[0]))
                else:
    #                 print(image_file[0])
                    out_file = os.path.join(map_images_folder, os.path.basename(image_file[0]))
#                     print(os.path.basename(image_file[0]))
                    flight_logs[video_ind].loc[ind_list[video_ind][map_frame_ind], 'image_name'] = os.path.basename(image_file[0])
                    if SAVE_IMAGES:
                        shutil.copyfile(image_file[0], out_file)
                break
            else:
                frame_num -= (fl_dict['last_frame'] + 2)
                ex_frame_num += (fl_dict['last_frame'] + 2) 


print(map_images_folder)

/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/processed-videos/raw-footage/observation088/map-images/whole-observation


In [29]:
# Save the dataframes

dataframe_folder = os.path.join(output_folder_root, 'drone-logs')
if not os.path.isdir(dataframe_folder):
    os.mkdir(dataframe_folder)
for flight_num in range(len(flight_logs)):
    filename = os.path.join(dataframe_folder, 'flight_{}.pkl'.format(flight_num))
    flight_logs[flight_num].to_pickle(filename)

In [30]:
for flight_ind in range(3):
    print(flight_logs[flight_ind].loc[flight_logs[flight_ind]['used_in_map'], ['image_name']])

                            image_name
980    APR08_2018_A_DJI_0005_06246.jpg
994    APR08_2018_A_DJI_0005_06330.jpg
1016   APR08_2018_A_DJI_0005_06462.jpg
3862   APR08_2018_A_DJI_0006_04164.jpg
3898   APR08_2018_A_DJI_0006_04392.jpg
3927   APR08_2018_A_DJI_0006_04590.jpg
3956   APR08_2018_A_DJI_0006_04776.jpg
3986   APR08_2018_A_DJI_0006_04968.jpg
4018   APR08_2018_A_DJI_0006_05166.jpg
4051   APR08_2018_A_DJI_0006_05364.jpg
4078   APR08_2018_A_DJI_0006_05526.jpg
4447   APR08_2018_A_DJI_0006_07778.jpg
4475   APR08_2018_A_DJI_0006_07952.jpg
4485   APR08_2018_A_DJI_0006_08012.jpg
5518   APR08_2018_A_DJI_0006_14330.jpg
5561   APR08_2018_A_DJI_0006_14588.jpg
5568   APR08_2018_A_DJI_0006_14630.jpg
5735   APR08_2018_A_DJI_0006_15650.jpg
5762   APR08_2018_A_DJI_0006_15818.jpg
6247   APR08_2018_A_DJI_0006_18752.jpg
6274   APR08_2018_A_DJI_0006_18914.jpg
6289   APR08_2018_A_DJI_0006_19016.jpg
7838   APR08_2018_A_DJI_0007_09004.jpg
7874   APR08_2018_A_DJI_0007_09226.jpg
7912   APR08_2018_A_DJI_0

In [31]:
dfs = []
for flight_ind in range(3):
    dfs.append(flight_logs[flight_ind].loc[ind_list[flight_ind], ['used_in_map', 'latitude', 'longitude', 'altitude(feet)']].copy())
total_df = pd.concat(dfs, ignore_index=True)

In [33]:
total_df.to_csv(os.path.join(output_folder_root, 'map-images/whole_obs_file_lat_long_alt.csv'), header=False, index=False)